# Get DGIDB and MSigDB Results from Outlier Gene Lists

Several lists of genes are generated in the outlier analyis step.
For each of these lists:
    - load the DGIDB drug list and get drugs which have a known interaction with these genes
    - load the MSIGDB pathway lists (referred to as GSEA in this script) and compute overlaps
        with gene sets
        
Inputs:
    - outlier results (step 4.0)
    - dgidb gene-to-drug list
    - msigdb pathway and gene lists and pathway/gene descriptions
    
Output json:
    - genelists: pc_up, pd_up, top5, comm_up, pc_down, pd_down
    - dgidb_results: pc_up, pd_up, top5, comm_up
    - gsea_results: pc_up, pd_up, top5, comm_up

In [ ]:
import sys
import os
import pandas as pd
import time
import json
import logging
import tempfile

# Setup: load conf, retrieve sample ID, logging
with open("conf.json","r") as conf:
    c=json.load(conf)
sample_id = c["sample_id"]    
print("Running on sample: {}".format(sample_id))

logging.basicConfig(**c["info"]["logging_config"])
logging.info("\n5: Get DGIDB and MSIGDB Info")
def and_log(s):
    logging.info(s)
    return s

# Load the outlier results dataframe
with open(c["json"]["4.0"],"r") as jf:
    outlier_results_df = pd.DataFrame.from_dict(
        json.load(jf)["outlier_results"],
    orient="columns",
    dtype="float64")

# List of Hugo genes -> DGIDB Drugs
with open(c["ref_file"]["dgidb_genes_and_drugs"], "r") as f:
    hugo_gene_to_drugs = json.load(f)
    
j = {}

In [ ]:
### Functions ###

# Takes: outlier result dataframe
# loads it up as a dataframe and returns the following:
# - pc up genes  - Dataframe
# - pd up genes  -  Dataframe
# - top5% up genes - Dataframe
# - common (intersection of PC and PD up ) - Index
def get_genelists(oa_res):
    
    genelists = {}
    
    genelists["pc_up"] = oa_res.loc[oa_res["pc_outlier"] == "pc_up"][["sample","pc_median"]]
    genelists["pd_up"] = oa_res.loc[oa_res["pd_outlier"] == "pd_up"][["sample","pd_median"]]
    genelists["top5"] = oa_res.loc[oa_res["is_top_5"] == "top5"][["sample"]]

    # common 
    genelists["comm_up"] = genelists["pc_up"].index.intersection(genelists["pd_up"].index)    
    
    genelists["pc_down"] = oa_res.loc[oa_res["pc_outlier"] == "pc_down"][["sample","pc_median"]]
    genelists["pd_down"] = oa_res.loc[oa_res["pd_outlier"] == "pd_down"][["sample","pd_median"]]
    
    return genelists

# Takes : array of hugo gene names
# returns a dictionary {druggable genes : [drugs for that gene] }
# Keys are the provided Hugo gene name rather than DGIDB's interpretation of that gene
# hugo_gene_to_drugs is dict with format {myGene :  (digdbGene, [list of drugs])}
def get_dgidb(genelist):
    druggable_genes = dict((k, hugo_gene_to_drugs[k][1]) for k in genelist if k in hugo_gene_to_drugs)
    return druggable_genes

In [ ]:
# Make genelists and add to the json.
# Within the json: genelist comm_up is an array of genes
# The other genelists are a dict with keys [sample, pc_median],
# each of which is a dict with keys = gene & value = expression for that gene

j["genelists"] = {}

genelists = get_genelists(outlier_results_df)

def make_str(x): # format floats but leave strings alone.
    if(type(x) == float):
        return "%.12g" % x
    return x

for listname, genes in genelists.iteritems():
    if type(genes) == pd.core.frame.DataFrame:
        # Don't applymap if the DF is empty - see https://github.com/pandas-dev/pandas/issues/8222
        if genes.empty:
            genes_df_stringified = genes
        else:
            genes_df_stringified = genes.applymap(make_str)
        j["genelists"][listname] = json.loads(genes_df_stringified.to_json(orient='columns'))
    elif type(genes) == pd.indexes.base.Index:
        j["genelists"][listname] = list(genes)

In [ ]:
# Get the dgidb results
print( "\nGetting DGIDB for {}".format(sample_id))

j["dgidb_results"] = {}
clean_genelists = {} # Dict of Index objects that contain the gene lists

# run dgidb and accumulate results in the json
# Also accumulate "cleaned" genelists for gsea input
for gl in ["top5", "comm_up", "pc_up", "pd_up"] :
    sys.stdout.write("{}[{}]".format(gl, len(genelists[gl])))
    sys.stdout.flush()

    if gl == "comm_up":
        clean_genelists[gl] = genelists[gl]
    else:
        clean_genelists[gl] = genelists[gl].index

    j["dgidb_results"][gl] = get_dgidb(clean_genelists[gl]) # DGIDB call

    sys.stdout.write(", dgidb.\n")
    sys.stdout.flush()

In [ ]:
# Set up the tmpdir to run the gsea script
workdir = tempfile.mkdtemp(dir=c["dir"]["temp"])
logging.info("Step 5: Input files for R stored in: {}".format(workdir))
workdir

In [ ]:
# Input file for gsea script - one file containing all genelists
# First item in each row is the geneset name and all remaining are the genes in that set
# Track which lists have no genes in them - gsea will skip these

genelist_filepath = os.path.join(workdir, "genelists.tsv")
empty_genelists = []
with open(genelist_filepath, "w") as f:
    for listname, index_of_genes in clean_genelists.iteritems():
        genelist_line = "{}\t{}\n".format(listname, "\t".join(index_of_genes))
        f.write(genelist_line)
        if not index_of_genes.any():
            empty_genelists.append(listname)
    f.write("\n")

In [ ]:
# some abbreviated args to shorten the R script argument line --
# these will get reparsed immediately into new variable names
glf = genelist_filepath
hpath = c["ref_file"]["h_symbols_pathway"]
cpath = c["ref_file"]["c2_cp_symbols_pathway"]
mspath = c["ref_file"]["msigdb_symbols_pathway"]
hdesc = c["ref_file"]["h_symbols_pathway_description"]
cdesc = c["ref_file"]["c2_cp_symbols_pathway_description"]
gdesc = c["ref_file"]["hgnc_gene_description"]
suffix = "_gsea_results.txt"

In [ ]:
%%script Rscript - "$glf" "$hpath" "$cpath" "$mspath" "$hdesc" "$cdesc" "$gdesc" "$suffix"

# input file:
# One row for each outlierList
# first item is geneset name, all remaining are genes

# Expand the variable names back out. Main is  below.
args<-commandArgs(TRUE)

geneListsFile          <-args[1]
h.all.v6.1.symbols.gmt <- args[2]
c2.cp.v6.1.symbols.gmt <- args[3]
msigdb.v6.1.symbols.gmt <- args[4]
h.all.v6.1.symbols.DESCRIPTIONS.gmt <- args[5]
c2.cp.v6.1.symbols.DESCRIPTIONS.gmt <- args[6]
HGNC_gene_descriptions <- args[7]
outputFileSuffix <- args[8]


# Import necessary libraries
library(plyr)
library(tidyverse)

### Import files into environment


# Given a row of "listname \t gene1 \t gene2 ..."
# return a list with items (listName, genes)
parseGeneRow <- function(row){
    result <- list()
    parsedRow <- strsplit(row, "\t")[[1]]
    result$listName <- parsedRow[1]
    if(length(parsedRow) < 2){
        result$genes <- NA
    }
    else{
        result$genes <- parsedRow[2:length(parsedRow)]
    }
    return(result)
}

# Load the hallmark pathways, genes, descriptions
# returns a list with items allPathwaysDF, testTotalGenes, allDescDF
setupGSEA <- function(hallmarkFile, cpFile, geneUniverseFile, hallmarkDescFile, cpDescFile, geneDescFile){

    # Scan our hallmark .gmt file containing hallmark pathways and genes
    hallmarkPathwaysRaw=scan(hallmarkFile, what="list", sep="\n")
    hallmarkPathwaysMatrixList=lapply(hallmarkPathwaysRaw, function(x) { y1=strsplit(x, "\t")[[1]]; y2= y1[3:length(y1)];matrix(data=c(rep(y1[1], length=length(y2)), y2), ncol=2, byrow=FALSE)})
    hallmarkPathwaysDF <-data.frame(do.call("rbind", hallmarkPathwaysMatrixList))
    colnames(hallmarkPathwaysDF)=c("GeneSet", "Gene")

    # Scan our CP .gmt file containing all canonical pathways and genes
    canonicalPathwaysRaw=scan(cpFile, what="list", sep="\n")
    canonicalPathwaysMatrixList=lapply(canonicalPathwaysRaw, function(x) { y1=strsplit(x, "\t")[[1]]; y2= y1[3:length(y1)];matrix(data=c(rep(y1[1], length=length(y2)), y2), ncol=2, byrow=FALSE)})
    canonicalPathwaysDF <-data.frame(do.call("rbind", canonicalPathwaysMatrixList))
    colnames(canonicalPathwaysDF)=c("GeneSet", "Gene")

    # Combine hallmark and canonical pathways
    allPathwaysDF = bind_rows(hallmarkPathwaysDF, canonicalPathwaysDF)

    # Import Universe of Genes
    universePathwaysRaw=scan(geneUniverseFile, what="list", sep="\n")
    universePathwaysMatrixList=lapply(universePathwaysRaw, function(x) { y1=strsplit(x, "\t")[[1]]; y2= y1[3:length(y1)];matrix(data=c(rep(y1[1], length=length(y2)), y2), ncol=2, byrow=FALSE)})
    universePathwaysDF <-data.frame(do.call("rbind", universePathwaysMatrixList))
    colnames(universePathwaysDF)=c("GeneSet", "Gene")
    testTotalGenes = subset(universePathwaysDF, !is.na(Gene) & !duplicated(Gene))


    #write_tsv(testTotalGenes, file.path("./", "testTotalGenes")) # TEMP REMOVE ME


    # Load the geneset descriptions
    hallmarkDescriptions <- as.list(scan(hallmarkDescFile, what="list", sep="\n"))
    cpDescriptions <- as.list(scan(cpDescFile, what="list", sep="\n"))
    allDescriptions <- c(hallmarkDescriptions, cpDescriptions)
    allDescMatrixList = lapply(allDescriptions, function(x) {strsplit(x, "\t")[[1]]})
    allDescDF <- data.frame(do.call("rbind", allDescMatrixList), stringsAsFactors=FALSE)
    colnames(allDescDF) <- c("GeneSet", "Description")

    # load the gene descriptions
    # Columns: Approved symbol Approved name   NCBI gene ID    HGNC ID
    geneDescriptions <- as.list(scan(geneDescFile, what="list", sep="\n"))
    geneDescMatrixList = lapply(geneDescriptions, function(x) {strsplit(x, "\t")[[1]]})
    geneDescDF <- data.frame(do.call("rbind", geneDescMatrixList), stringsAsFactors=FALSE)
    colnames(geneDescDF) <- c("Approved symbol", "Approved name", "NCBI gene ID", "HGNC ID")
        
    # Final result
    result <- list()
    result$allPathwaysDF <- allPathwaysDF
    result$testTotalGenes <- testTotalGenes
    result$allDescDF <- allDescDF
    result$geneDescDF <- geneDescDF
    return(result)
}

# Run gsea algorithm
# takes: outlier list; all genes in universe; all potential pathways of interest; pathway descriptions
gsea <- function(outputFileSuffix, pc_upoutliers, testTotalGenes, allPathwaysDF, allDescDF, geneDescDF, outputFilePrefix ){


    # For concordance, when testing: Reject the extra genes that the website doesn't like
    # Don't run this code in production; we'd actually prefer to keep the extra genes if we can
    # reject_these <- read.table("genes_to_reject.txt", stringsAsFactors=FALSE)
    # pc_upoutliers <- setdiff(pc_upoutliers, reject_these$V1) 


    # Where the nUpOutliers comes from
    overlap_upoutliers = intersect(pc_upoutliers, testTotalGenes$Gene)

    # Set up variables for Hypergeometric Distribution
    nTotalGenes = 45956 # Taken from output (results.txt) file
    nUpOutliers = length(overlap_upoutliers)

    ### Run tests and create output tables

    # Get our results by using Hypergeometric Distribution
    geneDF <- lapply(unique(allPathwaysDF$GeneSet), function(targetPathway){
      pathway_genes <- filter(allPathwaysDF, GeneSet == targetPathway)
      intersectGenes = intersect(overlap_upoutliers, pathway_genes$Gene)

      # Set up variables for Hypergeometric Distribution
      nIntersect = length(intersectGenes)
      nPathway = length(pathway_genes$Gene)

      # Sum up all probabilities for Hypergeometric Distribution
      p_value <- sum(dhyper(nIntersect:nUpOutliers, nPathway, nTotalGenes - nPathway, nUpOutliers))

      # Get description
      pathwayDescription = allDescDF[allDescDF$GeneSet == targetPathway, "Description"]
      if (length(pathwayDescription) == 0){
          pathwayDescription <- "MISSING DESCRIPTION"
      }

      # Create output list
      data_frame("Gene Set Name" = targetPathway, "# Genes in Gene Set (K)" = nPathway, 
                 "Description" = pathwayDescription, "# Genes in Overlap (k)" = nIntersect,
                 "k/K" = nIntersect/nPathway, "p-value" = p_value)
    }) %>% bind_rows()

    # calculate the FDR q-value. 
    # From http://software.broadinstitute.org/gsea/msigdb/help_annotations.jsp#overlap:
    # This is the false discovery rate analog of hypergeometric p-value after correction for multiple hypothesis testing according to Benjamini and Hochberg.
    # gene sets are considered enriched if the FDR q value is less than 0.05

    # Attach FDR q-value to our output table
    geneDF$"FDR q-value" <- p.adjust(geneDF$`p-value`, "BH")

    # Get the top 100 significantly enriched pathways
    significantly_enriched_pathways <- geneDF[order(geneDF$`FDR q-value`), ] %>%
      filter(`FDR q-value` <= 0.05) %>%
      head(n=100)

    # Truncate the numbers to 4 significant figures
    significantly_enriched_pathways$`k/K` <- signif(significantly_enriched_pathways$`k/K`, 4)
    significantly_enriched_pathways$`p-value` <- signif(significantly_enriched_pathways$`p-value`, 4)
    significantly_enriched_pathways$`FDR q-value` <- signif(significantly_enriched_pathways$`FDR q-value`, 4)

    # Create table of pathways found in our up_outlier genes
    gene_table <- data_frame(EGI = "1", GS = overlap_upoutliers, GD = "Gene Info")
    for (pathway in significantly_enriched_pathways$`Gene Set Name`){
      intersect_vector = intersect(overlap_upoutliers, filter(allPathwaysDF, GeneSet == pathway)$Gene)
      testInSet <- sapply(gene_table$GS, function(gene){
        if(gene %in% intersect_vector){
          return(pathway)
        }
        return("") # Gene not in pathway - leave blank
      })
      gene_table <- mutate(gene_table, hold = testInSet)
      names(gene_table)[length(names(gene_table))] <- pathway
    }
    
    # Check if gene_table needs sorting
    # Sorting is done based on rank of significantly enriched pathway, in reverse
    if (length(gene_table) >= 3){
      gene_table <- arrange_(gene_table, .dots = names(gene_table[3: length(gene_table)]))
    gene_table <-  gene_table[dim(gene_table)[1]:1,]
    }
    
    # Set column names for gene_table
    colnames(gene_table)[1:3] <- c("Entrez Gene Id", "Gene Symbol", "Gene Description")
    
    # Populate description and entrez gene id from geneDescDF
    gene_table$`Gene Description` <- geneDescDF$`Approved name`[match(
        gene_table$`Gene Symbol`, geneDescDF$`Approved symbol`)]

    gene_table$`Entrez Gene Id` <- geneDescDF$`NCBI gene ID`[match(
        gene_table$`Gene Symbol`, geneDescDF$`Approved symbol`)]

    ### Output data
    outputFileName <- file.path("./", paste0(outputFilePrefix, outputFileSuffix))

    # Header:
    headerText<-paste0("\nOverlap Results\n\nCollection(s):\tCP, H", # Static - CP and H
                       "\n# overlaps shown:\t", length(significantly_enriched_pathways$`Gene Set Name`),
                       "\n# genesets in collections:\t", "1379", # Static - count H + count CP
                       "\n# genes in comparison (n):\t", nUpOutliers,
                       "\n# genes in universe (N):\t", nTotalGenes, "\n" )
  
    write(headerText, outputFileName) 

    # Pathways
    #  no pathways = no output
    print(length(significantly_enriched_pathways$`Gene Set Name`))

    if( length(significantly_enriched_pathways$`Gene Set Name`) == 0){
        write("No overlaps found\n", outputFileName, append=TRUE)
    } else {
        write_tsv(significantly_enriched_pathways, outputFileName, col_names=TRUE, append=TRUE)

        # Genes by Pathway
        spacer <-"\n\nGene/Gene Set Overlap Matrix\n"
        write(spacer, outputFileName, append=TRUE)
        write_tsv(gene_table, outputFileName, append=TRUE, col_names=TRUE)
   } 
}

### MAIN ###

pathwayResources <- setupGSEA(h.all.v6.1.symbols.gmt,
                              c2.cp.v6.1.symbols.gmt,
                              msigdb.v6.1.symbols.gmt,
                              h.all.v6.1.symbols.DESCRIPTIONS.gmt,
                              c2.cp.v6.1.symbols.DESCRIPTIONS.gmt,
                              HGNC_gene_descriptions)

geneLists = scan(geneListsFile, sep="\n", what="list")
for( row in geneLists) {
    outlierList <- parseGeneRow(row)
    if( is.na(outlierList$genes) ){
        # No Genes in this list
        print(paste("Gene list was empty -- skipping gsea: ",  outlierList$listName))
    } else {
        print(paste("Running gsea: ",  outlierList$listName))
        gsea(
            outputFileSuffix,
            outlierList$genes,
            pathwayResources$testTotalGenes,
            pathwayResources$allPathwaysDF, 
            pathwayResources$allDescDF,
            pathwayResources$geneDescDF,
            outlierList$listName
            )
    }
}

In [ ]:
# Load the output files into the json
# Note: This does not cover the unlikely case where we expect no outputfile due to 
# empty genelist and one shows up anyhow.
j["gsea_results"] = {}

for listname in clean_genelists.keys():
    gsea_file = "{}_gsea_results.txt".format(listname)
    try:
        with open(gsea_file, "r") as f:
            j["gsea_results"][listname] = f.read()
    except IOError as e:
        if listname in empty_genelists:
            print "Got expected empty gsea results for {} - no genes!".format(listname)
            j["gsea_results"][listname] = ""
        else:
            print "Unexpected empty gsea results for {}!".format(listname)
            raise e

In [ ]:
# and delete the tmpdir and tmp input file
print("Deleting temp input file {}".format(genelist_filepath))
os.remove(genelist_filepath)
print("Deleting temp dir {}".format(workdir))
os.rmdir(workdir)

In [ ]:
with open(c["json"]["5"], "w") as jsonfile:
    json.dump(j, jsonfile, indent=2)
print("Done!")